# The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2019 Semester 1
-----
## Project 1: Gaining Information about Naive Bayes
-----
###### Student Name(s):
###### Python version:
###### Submission deadline: 1pm, Fri 5 Apr 2019

This iPython notebook is a template which you may use for your Project 1 submission. (You are not required to use it; in particular, there is no need to use iPython if you do not like it.)

Marking will be applied on the five functions that are defined in this notebook, and to your responses to the questions at the end of this notebook.

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find. 

In [1]:
# This function should open a data file in csv, and transform it into a usable format 
def preprocess(filename, header):
    dataset = []
    with open(filename) as csvfile:
        reader = csv.DictReader(csvfile, header)
        for row in reader:
            dataset.append(row)
    return dataset

In [69]:
# This function should build a supervised NB model
def train(dataset, header, classlabels):
    model = []
    smoothing = []
    classes = {}
    count = 0
    for index in range(len(dataset)):
        count += 1;
        if dataset[index]['class'] in classes:
            classes[dataset[index]['class']] += 1
        else:
            classes[dataset[index]['class']] = 1
    classes['count'] = count
    model.append(classes)
    
    #Laplace smoothing
    j = 0
    while j < len(header)-1:
        attributes = {}
        count = 0
        for index in range(len(dataset)):
            if dataset[index][header[j]] not in attributes:
                count += 1
                attributes[dataset[index][header[j]]] = 1
        j+=1
        attributes['count'] = count
        smoothing.append(attributes)
    i = 0
    
    while i < len(classlabels):
        current_class = classlabels[i]
        j = 0
        class_attributes = []
        while j < len(header)-1:
            attributes = smoothing[j].copy()
            for index in range(len(dataset)):
                if dataset[index]['class'] == current_class:
                    attributes['count'] += 1
                    attributes[dataset[index].get(header[j])] += 1
            class_attributes.append(attributes)
            j+=1
        model.append(class_attributes)
        i+=1
    return model

In [70]:
# This function should predict the class for an instance or a set of instances, based on a trained model 
def predict(model, dataset, classlabels, header):
    predicted_classes = []
    highest_probability = 0
    probability = 0
    for index in range(len(dataset)):
        row = dataset[index]
        highest_probability = 0
        probability = 0
        predicted_label = ''
        for label in classlabels:
            probability = model[0][label]/model[0]['count']
            for attribute in range(len(row)-1):
                probability *= model[classlabels.index(label)+1][attribute][row.get(header[attribute])]/model[classlabels.index(label)+1][attribute]['count']
            if probability > highest_probability:
                highest_probability = probability
                predicted_label = label
        predicted_classes.append(predicted_label)
    return predicted_classes

In [71]:
# This function should evaluate a set of predictions, in a supervised context 
def evaluate(predicted_classes, dataset):
    correct = 0
    wrong = 0
    count = 0
    for index in range(len(dataset)):
        row = dataset[index]
        if predicted_classes[index] == row['class']:
            correct += 1
        else:
            wrong += 1
        count += 1
    print("Accuracy is", correct/count*100,"% with", correct,"correct predictions and", wrong, "wrong predictions" )

In [72]:
def class_info(model, classlabels):
    classes = model[0]
    entropy_classes = 0
    i = 0
    while i < len(classes)-1:
        entropy_classes += classes[classlabels[i]]/classes['count']*math.log(classes[classlabels[i]]/classes['count'],2)
        i += 1
    entropy_classes = entropy_classes*(-1)
    return entropy_classes

In [73]:
def att_info(dataset, attribute):
    attributes = {}
    count = 0
    for index in range(len(dataset)):
        count += 1;
        if dataset[index][attribute] in attributes:
            attributes[dataset[index][attribute]] += 1
        else:
            attributes[dataset[index][attribute]] = 1
    attributes['count'] = count
    entropy_attributes = 0
    i = 0
    for value in attributes.keys():
        if value != 'count':
            entropy_attributes += attributes[value]/attributes['count']*math.log(attributes[value]/attributes['count'],2)
    entropy_attributes = entropy_attributes*(-1)
    return entropy_attributes

In [77]:
def attributes_info(dataset, header, model, classlabels):
    j = 0
    attributes_values = []
    while j < len(header)-1:
        count = 0
        attributes = []
        for index in range(len(dataset)):
            if dataset[index][header[j]] not in attributes:
                attributes.append(dataset[index][header[j]])
        j+=1  
        attributes_values.append(attributes)
    attribute_index = 0
    entropy_attributes = []
    print(attributes_values)
    for attribute_index in  range(len(header)-1):
        mean_info = 0
        for value in attributes_values[attribute_index]:
            i = 1
            j = 1
            total = 0
            entropy = 0
            while i <= len(classlabels):
                print("model",model[i][attribute_index][value])
                total += model[i][attribute_index][value]-1
                i += 1
            print(total)
            while j <= len(classlabels):
                if (model[j][attribute_index][value]-1)!= 0:
                    entropy += (model[j][attribute_index][value]-1)/total*math.log((model[j][attribute_index][value]-1)/total,2)
                j += 1
            entropy_attribute = entropy*(-1)
            mean_info += (total/model[0]['count'])*entropy_attribute
        entropy_attributes.append(mean_info)
    return entropy_attributes

In [78]:
# This function should calculate the Information Gain of an attribute or a set of attribute, with respect to the class
def info_gain(attributes_info,class_info, header):
    info_gain = []
    i = 0
    for value in attributes_info:
        info_gain.append(class_info-value)
    while i < len(header)-1:
        print("information gain for", header[i], "is", info_gain[i])
        i += 1
        

In [80]:
import csv
import math

#This changes depending on the file 
filename = 'nursery.csv'
header = 'parents','has_nurs','form','children','housing','finance','social','health','class'
classlabels = 'not_recom', 'recommend', 'very_recom', 'priority', 'spec_prior'



dataset = preprocess(filename, header)
model = train(dataset, header, classlabels)
predicted_classes = predict(model,dataset,classlabels, header)
evaluate(predicted_classes, dataset)
class_info1 = class_info(model, classlabels)
print(class_info1)
attributes = ['parents','has_nurs','form,children','housing','finance','social','health']
attributes_info1 = attributes_info(dataset, header, model, classlabels)
info_gain1 = info_gain(attributes_info1,class_info1, header) 
print(attributes_info1)
entropy_attribute = att_info(dataset, 'parents')
print(info_gain(attributes_info1,entropy_attribute,header))

Accuracy is 90.30092592592592 % with 11703 correct predictions and 1257 wrong predictions
1.7164959001837932
[['usual', 'pretentious', 'great_pret'], ['proper', 'less_proper', 'improper', 'critical', 'very_crit'], ['complete', 'completed', 'incomplete', 'foster'], ['1', '2', '3', 'more'], ['convenient', 'less_conv', 'critical'], ['convenient', 'inconv'], ['nonprob', 'slightly_prob', 'problematic'], ['recommended', 'priority', 'not_recom']]
model 1441
model 3
model 197
model 1925
model 759
4320
model 1441
model 1
model 133
model 1485
model 1265
4320
model 1441
model 1
model 1
model 859
model 2023
4320
model 865
model 3
model 131
model 1345
model 253
2592
model 865
model 1
model 133
model 1345
model 253
2592
model 865
model 1
model 67
model 905
model 759
2592
model 865
model 1
model 1
model 465
model 1265
2592
model 865
model 1
model 1
model 211
model 1519
2592
model 1081
model 3
model 119
model 1153
model 889
3240
model 1081
model 1
model 101
model 1093
model 969
3240
model 1081
model 1

Questions (you may respond in a cell or cells below):

1. The Naive Bayes classifiers can be seen to vary, in terms of their effectiveness on the given datasets (e.g. in terms of Accuracy). Consider the Information Gain of each attribute, relative to the class distribution — does this help to explain the classifiers’ behaviour? Identify any results that are particularly surprising, and explain why they occur.
2. The Information Gain can be seen as a kind of correlation coefficient between a pair of attributes: when the gain is low, the attribute values are uncorrelated; when the gain is high, the attribute values are correlated. In supervised ML, we typically calculate the Infomation Gain between a single attribute and the class, but it can be calculated for any pair of attributes. Using the pair-wise IG as a proxy for attribute interdependence, in which cases are our NB assumptions violated? Describe any evidence (or indeed, lack of evidence) that this is has some effect on the effectiveness of the NB classifier.
3. Since we have gone to all of the effort of calculating Infomation Gain, we might as well use that as a criterion for building a “Decision Stump” (1-R classifier). How does the effectiveness of this classifier compare to Naive Bayes? Identify one or more cases where the effectiveness is notably different, and explain why.
4. Evaluating the model on the same data that we use to train the model is considered to be a major mistake in Machine Learning. Implement a hold–out or cross–validation evaluation strategy. How does your estimate of effectiveness change, compared to testing on the training data? Explain why. (The result might surprise you!)
5. Implement one of the advanced smoothing regimes (add-k, Good-Turing). Does changing the smoothing regime (or indeed, not smoothing at all) affect the effectiveness of the Naive Bayes classifier? Explain why, or why not.
6. Naive Bayes is said to elegantly handle missing attribute values. For the datasets with missing values, is there any evidence that the performance is different on the instances with missing values, compared to the instances where all of the values are present? Does it matter which, or how many values are missing? Would a imputation strategy have any effect on this?

Don't forget that groups of 1 student should respond to question (1), and one other question of your choosing. Groups of 2 students should respond to question (1) and question (2), and two other questions of your choosing. Your responses should be about 150-250 words each.

In [ ]:
1."cmc" Accuracy is 50.577053632043445 % with 745 correct predictions and 728 wrong predictions
information gain for w-education is 0.07090633894894594
information gain for h-education is 0.04013859922938412
information gain for n-child is 0.10173991727554088
information gain for w-relation is 0.009820501434385065
information gain for w-work is 0.002582332379721608
information gain for h-occupation is 0.030474214560266777
information gain for standard-of-living is 0.032511460053806784
information gain for media-exposure is 0.01578645559562042

"car" Accuracy is 87.15277777777779 % with 1506 correct predictions and 222 wrong predictions
information gain for buying is 0.09644896916961399
information gain for maint is 0.07370394692148596
information gain for doors is 0.004485716626632108
information gain for persons is 0.2196629633399082
information gain for lug_boot is 0.030008141247605424
information gain for safety is 0.26218435655426386

"nursery" Accuracy is 90.30092592592592 % with 11703 correct predictions and 1257 wrong predictions
information gain for parents is 0.07293460750309921
information gain for has_nurs is 0.1964492804881155
information gain for form is 0.005572591715219621
information gain for children is 0.011886431475775616
information gain for housing is 0.01960202502287145
information gain for finance is 0.0043331270252000564
information gain for social is 0.022232616894017898
information gain for health is 0.958774960469976
